<a href="https://colab.research.google.com/github/anfalsiddiqui45/mental-health-models/blob/main/svm_tfidf_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay,
    accuracy_score,
    f1_score,
    precision_score,
    recall_score
)

import matplotlib.pyplot as plt


In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/train.csv")
test_df  = pd.read_csv("/content/drive/MyDrive/test.csv")


In [ ]:


X_train = train_df["statement"]
y_train = train_df["label"]

X_test = test_df["statement"]
y_test = test_df["label"]


In [ ]:
tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    max_features=50000,
    stop_words="english"
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf  = tfidf.transform(X_test)


In [ ]:
svm_model = LinearSVC(
    class_weight="balanced",
    max_iter=10000
)

svm_model.fit(X_train_tfidf, y_train)


In [ ]:
y_pred = svm_model.predict(X_test_tfidf)


In [ ]:
print(classification_report(
    y_test,
    y_pred,
    target_names=["Normal", "Depression", "Anxiety", "Stress"]
))


In [ ]:
metrics = {
    "accuracy": accuracy_score(y_test, y_pred),
    "macro_f1": f1_score(y_test, y_pred, average="macro"),
    "weighted_f1": f1_score(y_test, y_pred, average="weighted"),
    "macro_precision": precision_score(y_test, y_pred, average="macro"),
    "macro_recall": recall_score(y_test, y_pred, average="macro")
}

metrics


In [ ]:
cm = confusion_matrix(y_test, y_pred)

disp = ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=["Normal", "Depression", "Anxiety", "Stress"]
)

disp.plot(cmap="Blues")
plt.title("TF-IDF + Linear SVM")
plt.show()


In [ ]:
report = classification_report(
    y_test,
    y_pred,
    target_names=["Normal", "Depression", "Anxiety", "Stress"],
    output_dict=True
)

f1_scores = [report[c]["f1-score"] for c in report if c in ["Normal", "Depression", "Anxiety", "Stress"]]

plt.bar(["Normal", "Depression", "Anxiety", "Stress"], f1_scores)
plt.ylabel("F1-score")
plt.title("Per-Class F1 — TF-IDF + SVM")
plt.show()
